In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
#s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-14/hr=16/part-00000-76298f84-5414-49c5-88f6-a981ff06d94d-c000.csv .

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)
date_day = "2020-08-10"
hr = '20'

ap_neighbors_path = "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

ap_neighbors_path = s3_bucket + ap_neighbors_path
print(ap_neighbors_path)



# user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
# user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# # user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
# user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-08-10/hr=20/*.csv


In [2]:
rdd_ap_neigbhors = spark.sparkContext.textFile(ap_neighbors_path)
rdd_ap_neigbhors.take(3)

['site,ap,ap2,band,time_epoch,rssi',
 '0000063a-a4ce-4091-8921-c37560d46e47,d420b000ee16,d420b000ee07,5,1597092510,-68.0',
 '0000063a-a4ce-4091-8921-c37560d46e47,d420b000ee5c,d420b000ee7f,24,1597092493,-54.0']

In [3]:
# rdd_rrm_local.map(lambda line: (line.split(',')[0], line.split(',')[1])).collect()

In [4]:

df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(ap_neighbors_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

In [5]:
df_Schema

StructType(List(StructField(site,StringType,true),StructField(ap,StringType,true),StructField(ap2,StringType,true),StructField(band,IntegerType,true),StructField(time_epoch,IntegerType,true),StructField(rssi,DoubleType,true)))

In [6]:
df_scan.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|0000063a-a4ce-409...|d420b000ee16|d420b000ee07|   5|1597092510|-68.0|
|0000063a-a4ce-409...|d420b000ee5c|d420b000ee7f|  24|1597092493|-54.0|
|0000063a-a4ce-409...|d420b000ee70|d420b000ee20|  24|1597092511|-56.0|
|0000063a-a4ce-409...|d420b080eb69|d420b080eb37|   5|1597092479|-56.0|
|000cb16d-91d5-42e...|5c5b35505c85|        null|  24|1597092441|-74.0|
|000e9e7e-d400-4b2...|5c5b352e7328|5c5b352e73a0|   5|1597092495|-62.0|
|0014d8a6-dfa0-4f3...|d420b0804df3|d420b08030b1|   5|1597092510|-44.0|
|00193f17-4e86-476...|5c5b35aed886|5c5b35aed877|   5|1597092481|-74.0|
|00226d33-cd80-4b1...|5c5b35502003|5c5b35506d24|  24|1597092473|-61.0|
|00272816-8dc2-4f1...|5c5b357f32c7|5c5b35ae6df6|   5|1597092491|-72.0|
|00272816-8dc2-4f1...|5c5b357f3308|5c5b35ae6c89|  24|1597092176|-71.0|
|00272

In [7]:
df_scan.describe().show()

+-------+--------------------+------------+------------+------------------+--------------------+------------------+
|summary|                site|          ap|         ap2|              band|          time_epoch|              rssi|
+-------+--------------------+------------+------------+------------------+--------------------+------------------+
|  count|             3555639|     3555639|     3291690|           3555639|             3555639|           3555639|
|   mean|                null|        null|        null|16.190116319457626|1.5970923231360433E9|-66.08483763396677|
| stddev|                null|        null|        null| 9.348450748950679|   511.4168608160232| 8.199924671641448|
|    min|0000063a-a4ce-409...|5c5b350e01c7|5c5b350e0090|                 5|          1597089600|            -112.0|
|    max|fffca7ff-4d2f-455...|d420b0c12660|d420b0c12660|                24|          1597092774|             -11.0|
+-------+--------------------+------------+------------+----------------

In [8]:
df_scan.count()

3555639

In [9]:
site_id = "d1ee1d22-4b55-4c97-97c4-9d757144f45b"

In [10]:
df_scan_site = df_scan.filter(col('site')==site_id)

In [11]:
df_scan_site.count()

208

In [12]:
df_scan_site.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|d1ee1d22-4b55-4c9...|5c5b3552b70a|5c5b3552b796|   5|1597092480|-63.0|
|d1ee1d22-4b55-4c9...|5c5b3552b94e|5c5b3552b70a|   5|1597092474|-63.0|
|d1ee1d22-4b55-4c9...|5c5b3552c2fe|5c5b3552ba70|  24|1597092505|-67.0|
|d1ee1d22-4b55-4c9...|5c5b3552b660|5c5b3552b5e3|   5|1597092465|-58.0|
|d1ee1d22-4b55-4c9...|5c5b3552b94e|5c5b3552b5e3|  24|1597092474|-46.0|
|d1ee1d22-4b55-4c9...|5c5b3552b660|5c5b3552c2bd|  24|1597092465|-53.0|
|d1ee1d22-4b55-4c9...|5c5b3552c2bd|5c5b3552bb6f|  24|1597092504|-69.0|
|d1ee1d22-4b55-4c9...|5c5b3552b714|5c5b3552b94e|   5|1597092466|-58.0|
|d1ee1d22-4b55-4c9...|5c5b3552b94e|5c5b3552c2bd|  24|1597092474|-49.0|
|d1ee1d22-4b55-4c9...|5c5b3552b5e3|5c5b3552b7cd|  24|1597092476|-46.0|
|d1ee1d22-4b55-4c9...|5c5b3552b7cd|5c5b3552b94e|  24|1597092477|-57.0|
|d1ee1

# GraphFrames
The following command creates your first GraphFrame. The GraphFrame accepts two DataFrames as inputs—vertices and edges. GraphFrames like to have a naming convention in the column name, which you need to follow. Those rules are defined as follows:
* A DataFrame that represents vertices should contain a column named id. Here, personsDf contains a column name "id".

* A DataFrame that represents edges should contain columns named src and dst. Here, reationshipDf contains the columns "src" and "dst".

In [13]:
vertices= df_scan_site.selectExpr("ap as id").distinct()

In [14]:
edges=df_scan_site.selectExpr("ap as src", "ap2 as dst", "rssi as weight")

In [15]:
from graphframes import *
g = GraphFrame(vertices, edges)

In [16]:
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

+------------+
|          id|
+------------+
|5c5b3552c2fe|
|5c5b3552b796|
|5c5b3552ba70|
|d420b000a866|
|5c5b3552c2bd|
|5c5b3552bb6f|
|5c5b3552b70a|
|5c5b3552b714|
|5c5b3552b94e|
|5c5b3552b5e3|
|5c5b3552b7cd|
|5c5b3552b660|
+------------+

+------------+------------+------+
|         src|         dst|weight|
+------------+------------+------+
|5c5b3552b70a|5c5b3552b796| -63.0|
|5c5b3552b94e|5c5b3552b70a| -63.0|
|5c5b3552c2fe|5c5b3552ba70| -67.0|
|5c5b3552b660|5c5b3552b5e3| -58.0|
|5c5b3552b94e|5c5b3552b5e3| -46.0|
|5c5b3552b660|5c5b3552c2bd| -53.0|
|5c5b3552c2bd|5c5b3552bb6f| -69.0|
|5c5b3552b714|5c5b3552b94e| -58.0|
|5c5b3552b94e|5c5b3552c2bd| -49.0|
|5c5b3552b5e3|5c5b3552b7cd| -46.0|
|5c5b3552b7cd|5c5b3552b94e| -57.0|
|5c5b3552ba70|5c5b3552c2bd| -40.0|
|5c5b3552b5e3|5c5b3552b7cd| -62.0|
|5c5b3552b660|5c5b3552b70a| -58.0|
|5c5b3552b70a|5c5b3552b660| -58.0|
|5c5b3552b796|5c5b3552b714| -49.0|
|5c5b3552ba70|5c5b3552b714| -68.0|
|5c5b3552b660|5c5b3552b94e| -61.0|
|5c5b3552c2bd|5c5b3552b5

In [17]:
display(g)

GraphFrame(v:[id: string], e:[src: string, dst: string ... 1 more field])

In [18]:
# The following code gets the number of incoming links to Andrew. This is obtained by calling the inDegrees method.

g.degrees.filter("id = '5c5b3552b5e3'").show()

+------------+------+
|          id|degree|
+------------+------+
|5c5b3552b5e3|    43|
+------------+------+



In [19]:
# The following code shows how to get the number of links coming out from Andrew using the outDegrees method .

g.outDegrees.filter("id = '5c5b3552b5e3'").show()


+------------+---------+
|          id|outDegree|
+------------+---------+
|5c5b3552b5e3|       21|
+------------+---------+



In [20]:
sc.setCheckpointDir('graphframes_cps')


In [21]:
ranks = g.pageRank(resetProbability=0.10, maxIter=5)
# display(ranks.vertices.select("id","pagerank").orderBy(desc("pagerank")))


In [22]:
display(ranks.vertices)
display(ranks.edges)

DataFrame[id: string, pagerank: double]

DataFrame[src: string, dst: string, weight: double, weight: double]

In [26]:
vertices.show()

+------------+
|          id|
+------------+
|5c5b3552c2fe|
|5c5b3552b796|
|5c5b3552ba70|
|d420b000a866|
|5c5b3552c2bd|
|5c5b3552bb6f|
|5c5b3552b70a|
|5c5b3552b714|
|5c5b3552b94e|
|5c5b3552b5e3|
|5c5b3552b7cd|
|5c5b3552b660|
+------------+



In [35]:
# vertices.show()
# xap_s

Py4JJavaError: An error occurred while calling o57.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 247.0 failed 1 times, most recent failure: Lost task 4.0 in stage 247.0 (TID 7053, localhost, executor driver): org.apache.hadoop.fs.s3a.AWSS3IOException: getFileStatus on s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-08-10/hr=20/part-00000-ba6e840e-a010-4399-8105-4441601be503-c000.csv: com.amazonaws.services.s3.model.AmazonS3Exception: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: AF6ABB0E7C037941), S3 Extended Request ID: o1bWIJCf/VnAFWKfrsfHl7T8eI/5zwTxdkc0jycy3NkkK5B9XGWsVe1NkVLpyn22B4m1GfhIPbo=: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: AF6ABB0E7C037941)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:178)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3A

In [36]:
# Search from "Esther" for users of age < 32.

# ap1 = "5c5b3552b96c"
# ap2 = "5c5b3552b5e3"
# paths = g.bfs("id = ap1", "id = ap2")
# paths.show()

# # Specify edge filters or max path lengths.
# g.bfs("id = ap1", "id = ap2",\
#   edgeFilter="rssi > -75", maxPathLength=3)


In [ ]:
# g.connectedComponents()

result1 = g.stronglyConnectedComponents(maxIter=10)
result1.select("id", "component").orderBy("component").show()

In [ ]:
# result1.

In [ ]:
result2 = g.stronglyConnectedComponents(maxIter=10)
result2.select("id", "component").orderBy("component").show()

In [ ]:
result2

In [ ]:
date_day + hr

In [ ]:
s3_out_bucket = "s3://mist-test-bucket/" #top_1_time_epoch_by_site_ap_ap2_band/{}/{}/".format(date_day, hr)
s3_out_bucket_vertices = s3_out_bucket #+ "vertices/"
s3_out_bucket_edges = s3_out_bucket #+ "edges/"

print(s3_out_bucket_vertices, s3_out_bucket_edges)
g.vertices.write.parquet(s3_out_bucket_vertices)
g.edges.write.parquet(s3_out_bucket_vertices)
# s3_out_bucket


In [ ]:
# s3_out_bucket, rrm_local_path

In [ ]:
import networkx as nx
gp = nx.from_pandas_edgelist(edges.toPandas(),'src','dst')
nx.draw(gp, with_labels = True)

In [ ]:
g.persist()

In [ ]:
# graph.degrees.filter("id = 1").show()

In [ ]:
# GraphFrame provides the following built-in algorithms:
# Connected components

# Label propagation

# PageRank

# SVD++

# Shortest Path

# Strongly connected components

# Triangle count